In [1]:
import pandas as pd
import numpy as np
import requests

## read the JSON file that you saved in ex02

In [2]:
df = pd.read_json('../data/auto.json')

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


## enrich the dataframe using a sample from that dataframe

In [5]:
df_sample = df.sample(n=200, random_state=21).reset_index(drop=True)

In [6]:
df_sample.head()

,CarNumber,Refund,Fines,Make,Model
0,M0299X197RUS,2,"19,200.00",Ford,Focus
1,83298C154RUS,2,"8,594.59",Ford,Focus
2,H957HY161RUS,1,"2,000.00",Ford,Focus
3,T941CC96RUS,1,"2,000.00",Ford,Focus
4,H966HY161RUS,1,500.00,Ford,Focus


In [7]:
concat_rows = pd.concat([df, df_sample])

In [8]:
concat_rows.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


In [9]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
dtype: int64

## enrich the dataframe concat_rows by a new column with generated data

In [10]:
np.random.seed(21)
years = [np.random.randint(1980, 2020) for i in range(len(concat_rows))]
fines = concat_rows
fines['Year'] = years

In [11]:
fines.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1989
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,1995
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1984
3,X582HE161RUS,2,"2,000.00",Ford,Focus,2015
4,92918M178RUS,1,"5,700.00",Ford,Focus,2014


## enrich the dataframe by the data from another dataframe

In [12]:
owners = pd.read_json('../data/surname.json')

In [13]:
owners.columns = owners.iloc[0]

In [14]:
owners.drop(0, inplace=True)

In [15]:
owners.head()

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72


In [16]:
owners = owners[~owners.NAME.str.contains('\W')]
surname = [np.random.choice(owners.NAME) for i in range(len(fines))]

In [17]:
owners =  fines[['CarNumber']]

In [18]:
owners['SURNAME'] = surname

C:\Users\kruzh\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
owners.head()

,CarNumber,SURNAME
0,Y163O8161RUS,BAKER
1,E432XX77RUS,CRUZ
2,7184TT36RUS,MARTIN
3,X582HE161RUS,REED
4,92918M178RUS,COOPER


In [20]:
let = [chr(i) for i in range(ord('A'), ord('Z') + 1)] 
dig = [chr(i) for i in range(ord('0'), ord('9') + 1)]


def get_char(n, chars):
    res = ''
    for i in range(n):
        res += np.random.choice(chars)
    return res


def generate_car_num():
    res = np.random.choice(let)
    res += get_char(3, dig)
    res += get_char(2, let)
    res += get_char(3, dig)
    res += 'RUS'
    return res

In [21]:
random_fines = pd.DataFrame([fines[['Make', 'Model']].iloc[np.random.randint(0, len(fines))] for i in range(5)])

In [22]:
random_fines['CarNumber'] = [generate_car_num() for i in range(5)]
random_fines['Refund'] = [np.random.choice(fines['Refund']) for i in range(5)]
random_fines['Fines'] = [np.random.choice(fines['Fines']) for i in range(5)]
random_fines['Year'] = [np.random.choice(fines['Year']) for i in range(5)]

In [23]:
random_fines = random_fines[['CarNumber', 'Refund', 'Fines', 'Make', 'Model', 'Year']]

In [24]:
random_fines

,CarNumber,Refund,Fines,Make,Model,Year
310,C398HI259RUS,1,"2,000.00",Ford,Focus,2012
53,V794MU786RUS,2,"1,000.00",Ford,Focus,2004
722,Q314BL886RUS,2,"20,800.00",Ford,Focus,1999
667,F352NL670RUS,1,"1,400.00",Ford,Focus,2015
708,T573IB211RUS,1,"20,000.00",Ford,Focus,1983


In [25]:
fines = pd.concat([fines, random_fines])

In [26]:
owners = owners[:-20]

In [27]:
owners

,CarNumber,SURNAME
0,Y163O8161RUS,BAKER
1,E432XX77RUS,CRUZ
2,7184TT36RUS,MARTIN
3,X582HE161RUS,REED
4,92918M178RUS,COOPER
...,...,...
175,98047H178RUS,MENDOZA
176,X522OM161RUS,CHAVEZ
177,O136HO197RUS,ANDERSON
178,7089H8163RUS,SMITH


In [28]:
tmp = {'CarNumber': [generate_car_num() for i in range(3)],
        'SURNAME': [np.random.choice(owners.SURNAME) for i in range(3)]}

owners = pd.concat([owners, pd.DataFrame(tmp)])

In [29]:
result = owners.merge(fines, how='inner', on='CarNumber')
result.head()

,CarNumber,SURNAME,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,BAKER,2,"3,200.00",Ford,Focus,1989
1,Y163O8161RUS,BAKER,2,"1,600.00",Ford,Focus,1980
2,Y163O8161RUS,JIMENEZ,2,"3,200.00",Ford,Focus,1989
3,Y163O8161RUS,JIMENEZ,2,"1,600.00",Ford,Focus,1980
4,E432XX77RUS,CRUZ,1,"6,500.00",Toyota,Camry,1995


##  create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but only with all the years (the values can be different for you)

In [30]:
pivot = pd.pivot_table(result, values='Fines', index=['Make', 'Model'], 
                       columns=['Year'], aggfunc=np.sum)
pivot

Year                     1980         1981       1982       1983       1984  \
Make       Model                                                              
Ford       Focus   395,689.17 1,257,345.86 464,267.52 170,000.00 213,162.11   
           Mondeo         nan          nan        nan        nan        nan   
Skoda      Octavia   4,800.00          nan   7,700.00  23,189.17        nan   
Toyota     Camry    12,000.00    17,189.17        nan   7,200.00        nan   
           Corolla        nan          nan   4,000.00        nan        nan   
Volkswagen Golf     61,800.00          nan        nan  34,378.35     300.00   
           Jetta          nan          nan        nan        nan        nan   
           Passat         nan     8,000.00        nan  16,000.00  30,000.00   
           Touareg        nan          nan        nan        nan        nan   

Year                     1985       1986       1987       1988       1989  \
Make       Model                                                            
Ford       Focus   343,572.93 183,162.11 448,000.00 267,867.52 571,783.76   
           Mondeo         nan        nan        nan        nan  17,200.00   
Skoda      Octavia  29,183.76   1,200.00  15,600.00  15,600.00 274,200.00   
Toyota     Camry          nan        nan        nan        nan  44,800.00   
           Corolla        nan        nan  44,700.00        nan   8,000.00   
Volkswagen Golf     48,000.00        nan  84,900.00        nan   5,800.00   
           Jetta          nan        nan        nan        nan        nan   
           Passat   15,000.00  45,000.00  23,700.00        nan        nan   
           Touareg   5,800.00        nan        nan        nan        nan   

Year                ...       2010       2011       2012       2013  \
Make       Model    ...                                               
Ford       Focus    ... 383,518.80 271,735.04 182,800.00 395,983.76   
           Mondeo   ...        nan        nan  68,800.00        nan   
Skoda      Octavia  ...   8,800.00   1,000.00     500.00  19,594.59   
Toyota     Camry    ...        nan        nan  17,189.17        nan   
           Corolla  ...  72,000.00  25,783.76  60,600.00        nan   
Volkswagen Golf     ...        nan     600.00        nan  41,600.00   
           Jetta    ...        nan        nan        nan        nan   
           Passat   ...   7,800.00        nan        nan        nan   
           Touareg  ...   6,300.00        nan        nan        nan   

Year                     2014       2015       2016       2017       2018  \
Make       Model                                                            
Ford       Focus   370,629.62 496,300.00 218,162.11 902,800.00 623,767.52   
           Mondeo         nan        nan  46,200.00        nan        nan   
Skoda      Octavia     300.00 121,994.59     600.00  17,189.17 449,200.00   
Toyota     Camry     3,000.00        nan        nan        nan  43,189.17   
           Corolla        nan        nan   1,800.00  19,200.00  15,200.00   
Volkswagen Golf           nan   6,900.00        nan        nan   4,000.00   
           Jetta          nan        nan        nan        nan        nan   
           Passat         nan   1,200.00   2,100.00        nan        nan   
           Touareg   2,600.00     500.00        nan        nan        nan   

Year                     2019  
Make       Model               
Ford       Focus   170,072.93  
           Mondeo         nan  
Skoda      Octavia   9,500.00  
Toyota     Camry    18,100.00  
           Corolla        nan  
Volkswagen Golf           nan  
           Jetta          nan  
           Passat         nan  
           Touareg        nan  

[9 rows x 40 columns]

## save both dataframes fines and owners to csv files without index

In [31]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)